In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/방화벽로그분석_프로젝트

/content/drive/MyDrive/방화벽로그분석_프로젝트


In [15]:
import os, sys
import argparse
import time
import copy
from tqdm import tqdm

import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import numpy as np
import pandas as pd
from PIL import Image

from sklearn import preprocessing

import torch
from torch.autograd import Variable

from sklearn.model_selection import train_test_split
import csv

In [5]:
f = open('04_hashed.csv').read()
f.splitlines()[0:2]

['Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country',
 '20210410000018.64,154.58.159.102,103.177.12.42,6,52897,445,2,None,US']

In [27]:
def encoding(data_path):
  df = pd.read_csv(data_path, encoding='cp949', index_col=False)
  df = df.iloc[:, [1, 2, 6, 8]]
  df = df.drop_duplicates(subset=None, keep='first', inplace=False)

  # df['Action'].value_counts() # '2' 3251557, '1' 2543596, '0' 25157

  df_data = df[df['Action'] == 0]
  df_data.to_csv('data.csv')

encoding('04_hashed.csv')

In [28]:
df = pd.read_csv('label.csv', header=0, names=['src_ip','dst_ip','Action','dst_country'])
cls_name, _ = pd.factorize(df.Action, sort=True)

In [29]:
train, test_ = train_test_split(df, test_size=0.3)
test, val = train_test_split(test_, test_size=0.33)

print(len(train), len(test), len(val))

train.to_csv('train.csv')
test.to_csv('test.csv')
val.to_csv('val.csv')

2088032 599564 295308


In [31]:
train['Action'].value_counts()

2    2074027
1      14005
Name: Action, dtype: int64

In [9]:
class IPDataset(Dataset):

    """
    Args:
        csv_path (string): Path to the csv file with annotations.
        transform (callable, optional): Optional transform to be applied on a sample.
    """
    def __init__(self, csv_path):

        csv_path = os.path.expanduser(csv_path)
        assert os.path.exists(csv_path), '%s not found' % csv_path

        # read csv file
        self.df = pd.read_csv(csv_path, header=0, names=['src_ip','dst_ip','Action','dst_country'])
        self.df.cls_name, _ = pd.factorize(self.df.Action, sort=True)

        self._num_of_classes = 2
        self._len_of_dataset = len(self.df.cls_name)

        le = preprocessing.LabelEncoder() #convert string label to int
        le.fit(df.dst_country)
        self.dst_coun = le.transform(df.dst_country)

    def get_num_of_classes(self):
        return self._num_of_classes

    def __len__(self):
        return self._len_of_dataset

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        src_ip, dst_ip, action, dst_country  = self.df.iloc[idx]
        
        s_ip = src_ip.split('.')
        d_ip = dst_ip.split('.')

        dst_c = self.dst_coun[idx]

        data = s_ip + d_ip + [dst_c]
        label = action

        # normalization
        if action == 1:
          label = 0
        elif action == 2:
          label = 1

        data = [int(x) for x in data]

        data = np.array(data) 
        labels = np.array(label)

        # list -> numpy array -> torch array

        return torch.from_numpy(data), torch.from_numpy(labels)

In [10]:
train_dataset_csv = 'train.csv'
test_dataset_csv = 'test.csv'

IP_train_dataset = IPDataset(csv_path=train_dataset_csv)
IP_test_dataset = IPDataset(csv_path=test_dataset_csv)

for step, (data, label) in enumerate(IP_train_dataset):
    print(data, label)
    break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  from ipykernel import kernelapp as app


tensor([ 12, 150, 252, 150, 180, 184, 111,   1, 224]) tensor(1)


In [11]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, 32)
        self.linear2 = torch.nn.Linear(32, 16)
        self.linear3 = torch.nn.Linear(16, outputSize)
        self.s = nn.Sigmoid()
        # sigmoid -> [0,1]

    def forward(self, x):
        out = self.linear(x)
        out = self.linear2(out)
        out = self.linear3(out)
        out = self.s(out)
        return out

In [12]:
batch_size = 512

train_dataloader = DataLoader(IP_train_dataset, batch_size=batch_size, num_workers=8)
test_dataloader = DataLoader(IP_test_dataset, batch_size=batch_size, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
inputDim = 9 # takes variable 'x' 
outputDim = 1 # takes variable 'y'
learningRate = 0.001 
num_of_epochs = 3
batch_size = 512

model = linearRegression(inputDim, outputDim)

##### For GPU #######
if torch.cuda.is_available():
    model.cuda()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [16]:
for epoch in range(num_of_epochs):
  model.train()
  running_loss, running_corrects, total = 0, 0, 0

  p_bar = tqdm(total=len(train_dataloader))
  for step, (data, label) in enumerate(train_dataloader):

      data = data / 1.0 
      label = label / 1.0
      
      data = data.to(device)  
      label = label.to(device)

      optimizer.zero_grad()
      with torch.set_grad_enabled(True):
          # diff
          outputs = model(data)
          loss = criterion(outputs, label)
          loss.backward() #back prop
          optimizer.step()

      preds = (outputs>0.5).float()
      lb = torch.reshape(label, (data.size(0), 1))
      # print(label.data)
      # print(preds)
      # print(lb)

      s = torch.sum(preds == lb.data)  
      # print(s) 

      running_corrects += s

      total += data.size(0)
      running_loss += loss.item() * data.size(0)
      _loss = float(running_loss) / total
      _accu = float(running_corrects) / total * 100

      p_bar.set_description('[TRAIN on Epoch #{:d} Loss: {:.4f} acc: {:.2f}%]'.format(epoch + 1, _loss, _accu))
      p_bar.update(1)

  p_bar.close()

  model.eval()
  running_loss, running_corrects, total = 0, 0, 0
  p_bar = tqdm(total=len(test_dataloader))

  for step, (data, label) in enumerate(test_dataloader):

    data = data / 1.0 
    label = label / 1.0

    data = data.to(device)  
    label = label.to(device)
        
    with torch.set_grad_enabled(False):
      outputs = model(data)
      loss = criterion(outputs, label)
      preds = (outputs>0.5).float()
        
    # statistics
    lb = torch.reshape(label, (data.size(0), 1))
    # print(label.data)
    # print(preds)
    # print(lb)

    s = torch.sum(preds == lb.data)  
    # print(s) 

    running_corrects += s

    total += data.size(0)
    running_loss += loss.item() * data.size(0)
    _loss = float(running_loss) / total
    _accu = float(running_corrects) / total * 100

    p_bar.set_description('[TEST on Epoch #{:d} Loss: {:.4f} acc: {:.2f}%]'.format(epoch + 1, _loss, _accu))
    p_bar.update(1)
        
  p_bar.close()

  0%|          | 0/4079 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[TRAIN on Epoch #1 Loss: 0.0067 acc: 99.33%]: 100%|█████████▉| 4077/4079 [07:16<00:00, 11.72it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.

In [18]:
# Save the model checkpoint
torch.save(model.state_dict(), 'models/model.ckpt')

In [19]:
# validation

val_dataset_csv = 'val.csv'

IP_val_dataset = IPDataset(csv_path=val_dataset_csv)
val_dataloader = DataLoader(IP_val_dataset, batch_size=batch_size, num_workers=8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [20]:
model.eval()
running_loss, running_corrects, total = 0, 0, 0
p_bar = tqdm(total=len(val_dataloader))

for step, (data, label) in enumerate(val_dataloader):

  data = data / 1.0 
  label = label / 1.0

  data = data.to(device)  
  label = label.to(device)
      
  with torch.set_grad_enabled(False):
    outputs = model(data)
    loss = criterion(outputs, label)
    preds = (outputs>0.5).float()
      
  # statistics
  lb = torch.reshape(label, (data.size(0), 1))
  # print(label.data)
  # print(preds)
  # print(lb)

  s = torch.sum(preds == lb.data)  
  # print(s) 

  running_corrects += s

  total += data.size(0)
  running_loss += loss.item() * data.size(0)
  _loss = float(running_loss) / total
  _accu = float(running_corrects) / total * 100

  p_bar.set_description('[VAL on Epoch #{:d} Loss: {:.4f} acc: {:.2f}%]'.format(epoch + 1, _loss, _accu))
  p_bar.update(1)
      
p_bar.close()

  0%|          | 0/577 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[VAL on Epoch #3 Loss: 0.0065 acc: 99.35%]: 100%|█████████▉| 575/577 [00:59<00:00, 10.67it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:52

In [21]:
data_dataset_csv = 'data.csv'

IP_data_dataset = IPDataset(csv_path=data_dataset_csv)
data_dataloader = DataLoader(IP_data_dataset, batch_size=batch_size, num_workers=8)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [25]:
model.eval()
running_loss, running_corrects, total = 0, 0, 0
p_bar = tqdm(total=len(data_dataloader))

for step, (data, label) in enumerate(data_dataloader):

  data = data / 1.0 
  label = label / 1.0

  data = data.to(device)  
  label = label.to(device)
      
  with torch.set_grad_enabled(False):
    outputs = model(data)
    loss = criterion(outputs, label)
    preds = (outputs>0.5).float()
      
  # statistics
  lb = torch.reshape(label, (data.size(0), 1))
  # print(label.data)
  print(preds+1)
  # print(lb)

  s = torch.sum(preds == lb.data)  
  # print(s) 

  running_corrects += s

  total += data.size(0)
  running_loss += loss.item() * data.size(0)
  _loss = float(running_loss) / total
  _accu = float(running_corrects) / total * 100

  p_bar.set_description('[DATA on Epoch #{:d} Loss: {:.4f} acc: {:.2f}%]'.format(epoch + 1, _loss, _accu))
  p_bar.update(1)
      
p_bar.close()

  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([37])) that is different to the input size (torch.Size([37, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[DATA on Epoch #3 Loss: 1.0000 acc: 0.00%]: 100%|██████████| 1/1 [00:00<00:00,  2.92it/s]

tensor([[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]], device='cuda:0')


[DATA on Epoch #3 Loss: 1.0000 acc: 0.00%]: 100%|██████████| 1/1 [00:00<00:00,  1.75it/s]
